<a href="https://colab.research.google.com/github/DanielGreiner/Masterarbeit/blob/main/Berty_calcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Startup

In [1]:
#@title import libraries

import numpy as np
import pandas as pd
import pprint as pp

In [2]:
#@title global variables

dens_cat = 0.91 # g/cm^3

# Berty

## Berty Setup

In [3]:
d_R_berty = 0.01                    # m
r_R_berty = d_R_berty/2
A_R_berty = r_R_berty**2*np.pi      # m^2
L_R_berty = 0.1                     # m
V_R_berty = L_R_berty * A_R_berty   # m^3

In [4]:
V_dot_lab_sim_gas_vel = 11.62       # mlN/min
V_dot_demo = 1800*273/423           # ml/min
gas_vel = 0.00382                   # m/s

In [5]:
M_MeOH = 32.04                      # g/mol
M_H2O = 18.01                       # g/mol
Molar_volume_ideal_gas = 22414      # mlN/mol

total_1to1_flow_rate = 0.08/60      # mol/min
total_reactant_vol_flow_rate = total_1to1_flow_rate * Molar_volume_ideal_gas    # mlN/min


V_dot_N_min = 30
V_dot_N_min_plant = 18                    # mlN/min
total_liquid_flow_rate = V_dot_N_min + total_reactant_vol_flow_rate             # mlN/min
V_ratio_N = V_dot_N_min / total_liquid_flow_rate                                # 60% nitrogen, 40% methanol/water mixture by volumetric flowrates
V_dot_min = V_dot_N_min/V_ratio_N

total_liquid_flow_rate_min = V_dot_N_min_plant / V_ratio_N

safety = 1.2

total_reactant_liquid_mass = 100   # g
total_reaction_time_hr = 24         # h
total_reaction_time = total_reaction_time_hr * 60   # min

## mass finder for methanol water ratios

In [6]:
#@title mass finder function

def water_methanol_mass_required (ratio_methanol, ratio_water, V_flow_total):
    if V_flow_total < total_liquid_flow_rate_min:
        return print('Total flowrate too low. The minimum flow rate value is ' + str(round(total_liquid_flow_rate_min, 0)) + ' mlN/min. Set a higher value.')

    total_molar_flowrate_reactants = V_flow_total * (1 - V_ratio_N) / Molar_volume_ideal_gas      # mol / min with a safety of 20% added on top

    x_MeOH = ratio_methanol/(ratio_methanol + ratio_water)
    x_H2O = 1 - x_MeOH

    n_dot_MeOH = total_molar_flowrate_reactants * x_MeOH                                    # mol / min
    n_dot_H2O = total_molar_flowrate_reactants * x_H2O                                      # mol / min

    mass_flow_MeOH = n_dot_MeOH * M_MeOH                                                    # g / min
    mass_MeOH = mass_flow_MeOH * total_reaction_time * safety                               # g

    mass_flow_H2O = n_dot_H2O * M_H2O                                                       # g / min
    mass_H2O = mass_flow_H2O * total_reaction_time * safety                                 # g

    total_reactant_mass = mass_MeOH + mass_H2O                                              # g

    if total_reactant_mass > total_reactant_liquid_mass:
        return print ('The total reactant mass is greater than the provided liquid mass. The liquid will run out over the experimental time window. Increase maximum total liquid mass or decrease experimental time.')

    w_MeOH = mass_MeOH / total_reactant_mass                                                # mass % in reactant phase
    w_H2O = 1 - w_MeOH

    max_mass_MeOH = total_reactant_liquid_mass * w_MeOH
    max_mass_H2O = total_reactant_liquid_mass * w_H2O

    total_mass_flowrate = (mass_flow_MeOH + mass_flow_H2O) * 60

    if min_mass == True:
        return print('The mass methanol required is: ' + str(round(mass_MeOH, 2)) + 'g.\nThe mass water required is: '
                     + str(round(mass_H2O, 2)) + 'g.\nThe total mass flow rate of the reactant liquid is: ' + str(round(total_mass_flowrate, 3)) + ' g/h.')
    else:
        return print('The mass methanol required is: ' + str(round(max_mass_MeOH, 2)) + 'g.\nThe mass water required is: '
                     + str(round(max_mass_H2O, 2)) + 'g.\nThe total mass flow rate of the reactant liquid is: ' + str(round(total_mass_flowrate, 3)) + ' g/h.')

**water_methanol_mass_required**(ratio_methanol, ratio_water, V_flow_total [in mlN/min]) \\
Do you want to use the maximum mass of 1kg? --> min_mass = False \\
Do you want to use the minimum necessary mass for a given flow rate? --> min_mass = True \\
(safety of + 20%)

In [7]:
min_mass = False
water_methanol_mass_required(1, 4, 40)

The mass methanol required is: 30.78g.
The mass water required is: 69.22g.
The total mass flow rate of the reactant liquid is: 1.112 g/h.


In [8]:
total_liquid_flow_rate_min

35.931200000000004

Set partial pressure min to 0.15/0.15/0.7 for MeOH/H2O/N2 \\
increase MeOH or Water partial pressure \\
Nitrogen min = 20 mlN/min \\
Set flowrate to X. Lowest nitrogen % = 20/X \\
-> lowest flowrate is going to be 80 mlN/min \\
rest of the liquid is reactants at 1:4 \\
inputs = ratio methanol:water \\
outputs: nitrogen flowrate, liquid flowrate in g/h \\
save it as as dict of all flowrates, repeat for all \\
dont change the 1 -> 0.15 ratio for water/methanol \\
RTD monday if heating is ready


In [9]:
#@title mass flow finder function

def water_methanol_mass_flow_rate (ratio_methanol, ratio_water, V_flow_total, N_ratio_frac):
    if V_flow_total < total_liquid_flow_rate_min:
        return print('Total flowrate too low. The minimum flow rate value is ' + str(round(total_liquid_flow_rate_min, 0)) + ' mlN/min. Set a higher value.')

    total_molar_flowrate_reactants = V_flow_total * (1 - N_ratio_frac) / Molar_volume_ideal_gas      # mol / min with a safety of 20% added on top

    x_MeOH = ratio_methanol/(ratio_methanol + ratio_water)
    x_H2O = 1 - x_MeOH

    n_dot_MeOH = total_molar_flowrate_reactants * x_MeOH                                    # mol / min
    n_dot_H2O = total_molar_flowrate_reactants * x_H2O                                      # mol / min

    mass_flow_MeOH = n_dot_MeOH * M_MeOH                                                    # g / min

    mass_flow_H2O = n_dot_H2O * M_H2O                                                       # g / min

    total_mass_flowrate = (mass_flow_MeOH + mass_flow_H2O) * 60

    return round(total_mass_flowrate, 3)

In [10]:
#@title populate dict function

init_value_reactant = 0.15

partial_pressures_init = np.array([init_value_reactant, init_value_reactant, (1 - 2*init_value_reactant)])    # akin to x_i

def from_ratio_to_total_ratio(ratio_methanol, ratio_water):

    ratio_methanol_end = round(partial_pressures_init[0] * ratio_methanol, 2)
    ratio_water_end = round(partial_pressures_init[1] * ratio_water, 2)
    ratio_nitrogen_end = round(1 - (ratio_methanol_end + ratio_water_end), 2)

    ratio_array = [ratio_methanol_end, ratio_water_end, ratio_nitrogen_end]

    return ratio_array

def flow_rate_calc(flow_rate):

    ratio_cases = {'1:1', '2:1', '1:2', '1:3', '1:4'}

    flow_rate_dict = {_key : [from_ratio_to_total_ratio(int(_key[0]), int(_key[-1])),
                        [str(i * flow_rate) + ' mlN/min' for i in from_ratio_to_total_ratio(int(_key[0]), int(_key[-1]))],
                        str(water_methanol_mass_flow_rate(int(_key[0]), int(_key[-1]), flow_rate, from_ratio_to_total_ratio(int(_key[0]), int(_key[-1]))[-1])) + ' g/hr']
                        for _key in ratio_cases}

    return flow_rate_dict

In [11]:
#@title Create dictionary of flow rates for nitrogen. The goal is to keep nitrogen as low as possible

flowrate_array_eligible = np.array([80, 160, 250])                                      # volumetric flow rate in mlN/min

dict_flow_rates = {_key : flow_rate_calc(_key) for _key in flowrate_array_eligible}    # create filled, keyed dictionary


In [12]:
pp.pprint(dict_flow_rates)

{80: {'1:1': [[0.15, 0.15, 0.7],
              ['12.0 mlN/min', '12.0 mlN/min', '56.0 mlN/min'],
              '1.608 g/hr'],
      '1:2': [[0.15, 0.3, 0.55],
              ['12.0 mlN/min', '24.0 mlN/min', '44.0 mlN/min'],
              '2.186 g/hr'],
      '1:3': [[0.15, 0.45, 0.4],
              ['12.0 mlN/min', '36.0 mlN/min', '32.0 mlN/min'],
              '2.765 g/hr'],
      '1:4': [[0.15, 0.6, 0.25],
              ['12.0 mlN/min', '48.0 mlN/min', '20.0 mlN/min'],
              '3.343 g/hr'],
      '2:1': [[0.3, 0.15, 0.55],
              ['24.0 mlN/min', '12.0 mlN/min', '44.0 mlN/min'],
              '2.637 g/hr']},
 160: {'1:1': [[0.15, 0.15, 0.7],
               ['24.0 mlN/min', '24.0 mlN/min', '112.0 mlN/min'],
               '3.215 g/hr'],
       '1:2': [[0.15, 0.3, 0.55],
               ['24.0 mlN/min', '48.0 mlN/min', '88.0 mlN/min'],
               '4.373 g/hr'],
       '1:3': [[0.15, 0.45, 0.4],
               ['24.0 mlN/min', '72.0 mlN/min', '64.0 mlN/min'],
           